# Produccion electrica en base a la metereología

La idea principal del trabajo es ser capaces de predecir como va a afectar a que aumenten la produccion electrica de energias renovables las precipitaciones, el viento y las horas de sol.
Como las horas de sol y el viento son fenomenos cuya causa efecto es prácticamente inmediato vamos a centrarnos en la produccion de energia hidroelectrica.

En base a esto tenemos varias cuestiones que queremos despejar:
- Cuanto tarda en notarse en la generación eléctrica de fuentes de energía renovables los fenomenos metereológicos.
- Que cantidad de precipitaciones son necesarias para aumentar la generación de energia eléctica.
- Existe realmente una relación entre la metereologia y la producción electrica.


Como premisas partimos de :
- Vamos a considerar solo el poll de energia que proporciona Red Electrica de España (REE)
- Vamos a considerar que las empresas no trabajan bajo mala praxis y que intentan optimizar el uso de energias renovables.
- Se va a dividir el estudio por provincias, comunidades autonomas y a nivel estatal.

Como origenes de datos para el estudio vamos a utilizar los datos proporcionados por:
- Información de REE (https://www.ree.es/es/apidatos) obtenida mediante su API.
- Datos proporcionados por aemet(https://opendata.aemet.es/centrodedescargas/inicio), vamos a utilizar la libreria  aemet desarrollada por Pablo Moreno (https://pypi.org/project/python-aemet/).

Como requisitos para la ejecución del proyecto es necesario la instalación de la libreria python Aemet(pip install python-aemet)



In [39]:
from aemet import Aemet,Estacion
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import json
import urllib3

# Leemos la clave de la API de AEMET desde un fichero ubicado en la misma ruta que este notebook
with open('API_KEY_AEMET','r') as file:
    API_KEY_AE=file.read()

    

In [40]:
# Obtenemos la lista de estaciones de medicion de aemet 
info_estaciones=Estacion.get_estaciones(API_KEY_AE)

# Creamos un objeto Aemet para usar los metodos de la libreria aemet
aemet=Aemet(API_KEY_AE)


In [41]:
# Lectura de datos de AEMET

# Lectura de los datos por estacion y fecha (el resultado es una lista de diccionarios (json))



vcm=aemet.get_valores_climatologicos_diarios("2020-12-01T00:00:00UTC","2020-12-10T00:00:00UTC",'0252D')

# Pasamos los datos ha dataframes para su procesado

estaciones=pd.DataFrame(info_estaciones)
wheather_prov=pd.DataFrame(vcm)



In [70]:
# Obtenemos los datos de REE a traves de su API.

http=urllib3.PoolManager()

URL='https://apidatos.ree.es/es/datos/balance/balance-electrico?start_date=2019-01-01T00:00&end_date=2019-01-01T01:00&time_trunc=day'
request=http.request('GET',URL)

# Paso el resultado a str
type(request.data.decode())

     
# Paso el str a json para poder pasarlo a Dataframe
prueba=json.loads(request.data)

prueba




{'data': {'type': 'Balance de energía eléctrica',
  'id': 'bal1',
  'attributes': {'title': 'Balance de energía eléctrica',
   'last-update': '2020-01-10T08:15:42.000+01:00',
   'description': 'Balance eléctrico: asignación de unidades de producción según combustible principal. La producción neta de las instalaciones no renovables e hidráulicas UGH tiene descontados los consumos propios de las instalaciones. En dichos tipos de producción una generación negativa indica que la electricidad horaria consumida para los usos de las plantas excede la producción horaria de las instalaciones.'},
  'meta': {'cache-control': {'cache': 'HIT',
    'expireAt': '2021-01-19T08:36:22'}}},
 'included': [{'type': 'Renovable',
   'id': 'Renovable',
   'attributes': {'title': 'Renovable',
    'last-update': '2020-01-10T08:15:21.000+01:00',
    'description': None,
    'magnitude': None,
    'content': [{'type': 'Hidráulica',
      'id': '10288',
      'groupId': 'Renovable',
      'attributes': {'title': '

In [ ]:
# Definimos funciones que vamos a utilizar mas adelante


def estaciones_prov (prov,lista_estaciones):
    '''Data una provincia y una lista de diccionarios de estaciones de aemet. 
    Obtiene una lista de los ID de las estaciones de esa provincia.'''
    lista_id=[]
    for estacion in lista_estaciones:
        if estacion['provincia']==prov.upper():
            lista_id.append(estacion['indicativo'])
    return lista_id

help(estaciones_prov)